- Data input & preprocessing

In [ ]:
import keras
from keras.datasets import fashion_mnist 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans 

import plotly as py
import plotly.graph_objs as go
import plotly.express as px

(X_train,y_train), (X_test,y_test) = fashion_mnist.load_data()

#Reshapeing X to a 2D array
X = X_train.reshape(-1,X_train.shape[1]*X_train.shape[2])
y = y_train

- PCA

In [ ]:
#PCA
Clus_dataSet = StandardScaler().fit_transform(X) #(mean = 0 and variance = 1)

from sklearn.decomposition import PCA
# Make an instance of the Model
variance = 0.98
pca = PCA(variance)

pca.fit(Clus_dataSet)

Clus_dataSet = pca.transform(Clus_dataSet)

approximation = pca.inverse_transform(Clus_dataSet)

- K-Means

In [ ]:
k_means = KMeans(init = "k-means++", n_clusters = 10, n_init = 35)

k_means.fit(Clus_dataSet)

k_means_labels = k_means.labels_

G = len(np.unique(k_means_labels)) #Number of labels


#Hitech 2D matrix  for an array of indexes of the given label
cluster_index= [[] for i in range(G)]
for i, label in enumerate(k_means_labels,0):
    for n in range(G):
        if label == n:
            cluster_index[n].append(i)
        else:
            continue

Y_clust = [[] for i in range(G)]
for n in range(G):
    Y_clust[n]=y[cluster_index[n]] #Y_clust[0] contains array of answers for the cluster_index[0]
    assert(len(Y_clust[n])==len(cluster_index[n]))

#counts the number of each category in each cluster
def counter(cluster):
    unique, counts = np.unique(cluster, return_counts=True)
    label_index = dict(zip(unique, counts))
    return label_index

label_count= [[] for i in range(G)]
for n in range(G):
    label_count[n] = counter(Y_clust[n])

class_names = {0:'T-shirt/top', 1:'Trouser',2: 'Pullover',3: 'Dress',4: 'Coat',5:
               'Sandal',6: 'Shirt', 7:'Sneaker',8:  'Bag',9: 'Ankle boot'}

def plotter(label_dict):
    plt.bar(range(len(label_dict)), list(label_dict.values()), align='center')
    a = []
    for i in [*label_dict]: a.append(class_names[i])
    plt.xticks(range(len(label_dict)), list(a), rotation=45, rotation_mode='anchor')

plt.figure(figsize=(20,20))
for i in range (1,11):
    plt.subplot(5, 2, i)
    plotter(label_count[i-1]) 
    plt.title("Cluster" + str(i-1))


#3D Plotly Visualisation of Clusters using go

layout = go.Layout(
    title='<b>Cluster Visualisation</b>',
    yaxis=dict(
        title='<i>Y</i>'
    ),
    xaxis=dict(
        title='<i>X</i>'
    )
)

colors = ['red','green' ,'blue','purple','magenta','yellow','cyan','maroon','teal','black']
trace = [ go.Scatter3d() for _ in range(11)]
for i in range(0,10):
    my_members = (k_means_labels == i)
    index = [h for h, g in enumerate(my_members) if g]
    trace[i] = go.Scatter3d(
            x=Clus_dataSet[my_members, 0],
            y=Clus_dataSet[my_members, 1],
            z=Clus_dataSet[my_members, 2],
            mode='markers',
            marker = dict(size = 2,color = colors[i]),
            hovertext=index,
            name='Cluster'+str(i),
    #             line=dict(shape='spline')
            )

    #data.append(trace1)
fig = go.Figure(data=[trace[0],trace[1],trace[2],trace[3],trace[4],trace[5],trace[6],trace[7],trace[8],trace[9]], layout=layout)
    
py.offline.iplot(fig)